### This notebook includes an alternative version of the functions used in simulating the diauxic growth process: in this one it includes lags. 

In [6]:
import numpy as np
from math import *
import random
import itertools
from scipy.optimize import root
from scipy import stats
from scipy import special
import copy
import time
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import matplotlib.patches as mpatches
import seaborn as sns
import collections
import pickle
from sklearn.linear_model import LinearRegression
import matplotlib.gridspec as gridspec

In [ ]:
# functions involved in the serial dilution simulation

# functions involved in the serial dilution simulation
def dilute_check(system, growth_rate_list, lagtime = 0, ignore_this_run = False, ReturnInvasionChance = True):
    # t_points is the end of each nutrient's depletion, as well as 0 and T_dilution(period).
    t_points = []
    # c_points is the bug densities of each t point.
    c_points = []
    # r_points is the available nutrients at each t point.
    r_points = []
    #conc_points is the concentration of each nutrient.
    conc_points = []
    # inv_points is the measure of conflict or invasibility at each t point. 
    inv_points = []
    t_switch = 0
    t_points.append(t_switch)
    survivors, concent, pref_list, growth = output(system)
    c_points.append(concent)
    conc_points.append([i for i in system['res_concentration']])
    lag_flag = [0 for i in range(Nb)]
    lag_begins = [0 for i in range(Nb)]
    #list of resource in use for each consumer
    use = [0 for i in range(Nb)]
    while t_switch < T_dilute:
        a = [i for i in system['res_available']]
        r_points.append(a)
        # list of consumers for each resource
        consumer = [[] for i in range(Nr)]
        for i in range(Nb):
            # if all are depleted, bug still uses its least preferred nutrient
            while system['res_available'][preference_list[i//Size][use[i]]] < 1 and use[i] < Nr - 1: # when there's no current resource left
                use[i] = use[i] + 1 # goto next resource
                lag_flag[i] = 1 # start the lag
                lag_begins[i] = t_switch # and mark the starting point. Notice: lags do superposition with each other.
            if system['bug_available'][i] > 0:
                consumer[preference_list[i//Size][use[i]]].append(i)
        # find the earliest depleted resource
        t_dep = T_dilute - t_switch
        for i in range(Nr):
            if system['res_available'][i] > 0:
                def remain(t):
                    # S = c - sum(B0(e^gt-1)/Y)
                    return system['res_concentration'][i] - sum([system['bug_density'][j]*(exp(t*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i] if lag_flag[j] == 0])
                # a little bit tricky here...
                t_i = root(remain, T_dilute + 1).x[0]
                if t_i < t_dep:
                    t_dep = t_i
        for i in range(Nb):
            if lag_flag[i] == 1:
                t_i = lag_begins[i] + float(lagTime) - t_switch
                if t_i < t_dep:
                    t_dep = t_i
        # update the system according to this t_dep

        ResInUse = sum([int(i!=[]) for i in consumer])
        ResAvailable = sum(system['res_available'])
        if(ResAvailable!=0):
            inv_points.append(1-ResInUse/ResAvailable)
        t_switch = t_switch + t_dep
        temp_bug_density = [i for i in system['bug_density']]
        for i in range(Nb):
            if system['res_available'][preference_list[i//Size][use[i]]] > 0 and lag_flag[i] == 0:
                temp_bug_density[i] = system['bug_density'][i]*exp(growth_rate_list[i][preference_list[i//Size][use[i]]]*t_dep)
        #print(t_switch, system['bug_density'], lag_flag)
        for i in range(Nr):
            if system['res_available'][i] > 0:
                system['res_concentration'][i] = system['res_concentration'][i] - sum([system['bug_density'][j]*(exp(t_dep*growth_rate_list[j][i]) - 1)/yields_list[j][i] for j in consumer[i] if lag_flag[j] == 0])
                if system['res_concentration'][i] < c_threshold:
                    system['res_available'][i] = 0
        for i in range(Nb):
            system['bug_density'][i] = temp_bug_density[i]
        t_points.append(t_switch)
        survivors, concent, pref_list, growth = output(system)
        c_points.append(concent)
        conc_points.append([i for i in system['res_concentration']])
        for i in range(Nb):
            if lag_flag[i] == 1 and lag_begins[i] + float(lagTime) == t_dep + t_switch:
                lag_flag[i] = 0 
                lag_begins[i] = 0 # there might be multiple bugs doing their lags together, so need to screen everything again
    if(ignore_this_run == False):
        global details
        details['t_info'].append(t_points)
        details['bug_info'].append((survivors, c_points))
        details['res_left'].append(r_points)
        if('res_concent' in details):
            details['res_concent'].append(conc_points)
        if('round_idx' in details.keys()):
            details['round_idx'].append(details['round_idx'][-1]+1)
    return system, t_points, c_points, r_points
# you can modify the fluctuating environment here.
def move_to_new(system, fluc = []):
    global Res
    global details
    global movestep
    movestep += 1
    baseline = Res
    baseline = [Nr * i / sum(baseline) for i in baseline]
    system['bug_density'] = [(i*D > b_threshold)*i*D for i in system['bug_density']]
    system['bug_available'] = [1*(system['bug_density'][i] > b_threshold) for i in range(Nb)]
    system['res_concentration'] = [(system['res_concentration'][i]*D + baseline[i]) / (D + 1) for i in range(Nr)]
    system['res_available'] = [1*(system['res_concentration'][i] > c_threshold) for i in range(Nr)]
    Res = baseline
    details['res_begin'].append(baseline)
    return system

def invade(system, bug, growth_rate_list, mode=0):
    global details
    #print("one invasion")
    # (starting from a new flask)
    # then add invasive species
    system1 = copy.deepcopy(system)
    # then dilute till steady
    system1, t_points, c_points, r_points = dilute_check(system1, growth_rate_list, True)
    #system1, t_points, c_points, r_points = dilute_check(system1, growth_rate_list)
    accum = 0
    if(len(t_points)>2):
        t_begin = t_points[0]
        i = 0
        for idx in range(len(t_points[:-2])):
            switch_flag = 0 # if switch to next nutrient
            t_end = t_points[idx + 1]
            pref_bug = preference_list[bug//Size]
            while(i < len(pref_bug)-1 and r_points[idx][pref_bug[i]] < 1): 
                i += 1
                switch_flag = 1
                if(i == len(pref_bug) - 1):
                    break
            if(switch_flag == 1):
                if(t_begin == t_points[0]):
                    t_slot = t_end - t_begin
                else:
                    t_slot = t_end - t_begin - float(lagTime)
                if(t_slot > 0):
                    #print(i, preference_list[bug//Size])
                    accum += t_slot*growth_rate_list[bug][preference_list[bug//Size][i]]
                t_begin = t_end
        t_end = t_points[-1]
        if(t_begin == t_points[0]):
            t_slot = t_end - t_begin
        else:
            t_slot = t_end - t_begin - float(lagTime)
        if(t_slot > 0):
            accum += t_slot*growth_rate_list[bug][preference_list[bug//Size][i]]
    else:
        for idx in range(len(t_points[:-1])):
            t_slot = t_points[idx + 1] - t_points[idx]
            pref_bug = preference_list[bug//Size]
            i = 0
            while(r_points[idx][pref_bug[i]] < 0): i += 1
            accum += t_slot*growth_rate_list[bug][preference_list[bug//Size][i]]
    if accum > log(1/D) or sum(system['bug_available']) == 0 or mode == 1:
        system['bug_available'][bug] = 1
        system['bug_density'][bug] = b0
        i=0
        eqm = 0
        while(len(output(system)[0])>Nr or i < dilute_to_steady or eqm == 0):
            i=i+1
            system, t_points, c_points, r_points = dilute_check(system, growth_rate_list)
            # move to a new flask
            system = move_to_new(system, fluc=[])
            if(len(details['bug_info']) > 2):
                if(len(details['bug_info'][-1][0]) == len(details['bug_info'][-2][0])):
                    if max([abs(details['bug_info'][-1][1][1][i]-details['bug_info'][-2][1][1][i])/details['bug_info'][-2][1][1][i] for i in range(len(details['bug_info'][-2][1][1]))]) < 1e-3:
                        eqm = 1
    # it is necessary to do 2 runs here
        system, t_points, c_points, r_points = dilute_check(system, growth_rate_list)
        system = move_to_new(system, fluc=[])
        ext_list = [i for i, v in enumerate(system['bug_available']) if v==0]
        return system, ext_list
    else:
        ext_list = [i for i, v in enumerate(system1['bug_available']) if v==0]
        if('round_idx' in details.keys()):
            details['round_idx'][-1] += T_dilute
        return system, ext_list

def round_robin_invade(system, ext_list, growth_rate_list):
    for bug in ext_list:
        system, new_ext_list = invade(system, bug, growth_rate_list)
    return system, new_ext_list

def output(system):
    survivors = [i for i, v in enumerate(system['bug_available']) if v != 0]
    pref_list = []
    concent = []
    growth = []
    for i in survivors:
        pref_list.append(preference_list[i//Size])
        concent.append(system['bug_density'][i])
        growth.append(growth_rate_list[i])
    return survivors, concent, pref_list, growth

def MeasureOutput(output, resources, g):
    system = {'res_available': np.heaviside(resources, 0), 'res_concentration': [i for i in resources], 'bug_available': [0 for i in range(Nb)], 'bug_density': [0 for i in range(Nb)]} 
    for bug in output[0]:
        system['bug_available'][bug] = 1
        system['bug_density'][bug] = output[1][output[0].index(bug)]
    for i in range(1):
        system, p1, p2, p3 = dilute_check(system, g)
        baseline = resources
        baseline = [Nr * i / sum(baseline) for i in baseline]
        system['bug_density'] = [(i*D > b_threshold)*i*D for i in system['bug_density']]
        system['bug_available'] = [1*(system['bug_density'][i] > b_threshold) for i in range(Nb)]
        system['res_concentration'] = [(system['res_concentration'][i]*D + baseline[i]) / (D + 1) for i in range(Nr)]
        system['res_available'] = [1*(system['res_concentration'][i] > c_threshold) for i in range(Nr)]
    system, p1, p2, p3 = dilute_check(system, g)
    return p1, p2, p3